# Computer Vision: Distillation

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,pandas,sklearn,tensorflow -g

import re
import os
import sys
from time import time
import pickle
import pathlib
import itertools
from time import time
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import swifter
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import watermark

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

np.random.seed(42)

CPython 3.7.3
IPython 7.8.0

numpy 1.17.4
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.0.0
machine    : x86_64
processor  : i386
CPU cores  : 16
interpreter: 64bit
Git hash   : 529455e71c3fbe1904341a957cbe9f1267d1c067


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

import tensorflow as tf2
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Input, Activation, Maximum, ZeroPadding2D, concatenate, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

import tensorflow_model_optimization as tfmot

In [33]:
sys.path.append(os.pardir)
from src.tf_tools import *

## Reload model

In [8]:
RESHAPE_SIZE = (65, 65, 3)
MODEL_REF = "./models/model2_weights_final.h5"

PLANT_CLASSES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 
                 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 
                 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
CLASSES_DICT_NAMES = {name: k for k, name in zip(range(len(PLANT_CLASSES)), PLANT_CLASSES)}
CLASSES_DICT_NUM = {k: name for k, name in zip(range(len(PLANT_CLASSES)), PLANT_CLASSES)}
NUM_CATEGORIES = len(PLANT_CLASSES)

In [4]:
def conv_layer(feature_batch, feature_map, kernel_size=(3, 3), strides=(1,1), zp_flag=False):
    if zp_flag:
        zp = ZeroPadding2D((1,1))(feature_batch)
    else:
        zp = feature_batch
    conv = Conv2D(filters=feature_map, 
                  kernel_size=kernel_size, 
                  data_format = 'channels_last',
                  strides=strides)(zp)
    bn = BatchNormalization(axis=3)(conv)
    act = LeakyReLU(1/10)(bn)
    return act


def dense_set(inp_layer, n, activation, drop_rate=0.3):
    dp = Dropout(drop_rate)(inp_layer)
    dns = Dense(n)(dp)
    bn = BatchNormalization(axis=-1)(dns)
    act = Activation(activation=activation)(bn)
    return act


def multi_conv_model(img_shape=RESHAPE_SIZE, opt_name="Adam"):
    input_img = Input(shape=img_shape)

    # First stage of Conv and Max Pooling
    conv1 = conv_layer(input_img, 64)
    conv2 = conv_layer(conv1, 64, zp_flag=False)
    mp1 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(conv2)
    # Second stage of Conv and Max Pooling
    conv3 = conv_layer(mp1, 128)
    conv4 = conv_layer(conv3, 128)
    mp2 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(conv4)
    # Third stage
    conv7 = conv_layer(mp2, 256)
    conv8 = conv_layer(conv7, 256)
    conv9 = conv_layer(conv8, 256)
    mp3 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(conv9)
    # Apply dense layers
    flt = Flatten()(mp3)
    ds1 = dense_set(flt, 128, activation='tanh')
    out = dense_set(ds1, 12, activation='softmax')

    model = Model(inputs=input_img, outputs=out)
    
    # The first 50 epochs are used by Adam opt.
    # Then 30 epochs are used by SGD opt.
    if opt_name == "Adam":
        model_optimizer = Adam(lr=2 * 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    elif opt_name == "SGD":
        model_optimizer = SGD(lr=1 * 1e-1, momentum=0.9, nesterov=True)
    else:
        raise ValueError("Selected optimizer not supported")
    model.compile(loss='categorical_crossentropy',
                   optimizer=model_optimizer,
                   metrics=['accuracy'])
    # Display the model's architecture
    model.summary()
    return model

In [5]:
def get_callbacks(filepath=None, save_best=True, learning_schedule="Plateau", patience=6):
    callbacks = []
    
    if learning_schedule == "Plateau":
        # Reduce learning rate when a metric has stopped improving
        lr_schedule = ReduceLROnPlateau(monitor='val_acc', 
                                      factor=0.1, 
                                      min_delta=1e-5, 
                                      patience=patience, 
                                      verbose=1)
    elif learning_schedule == "Annealer":
        # Decrease each epoch
        lr_schedule = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=1)
    
    callbacks.append(lr_schedule)
    
    if filepath:
        # Save the model after every epoch
        msave = ModelCheckpoint(filepath, save_best_only=save_best)
        callbacks.append(msave)
        
    return callbacks

In [6]:
model = multi_conv_model(opt_name="Adam")
model.load_weights(MODEL_REF)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 65, 65, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 63, 63, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 63, 63, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 61, 61, 64)        0     

In [34]:
# Save complete model
COMPLETE_MODEL_PATH = "./models/model2_complete"
COMPLETE_MODEL_FN = "model2"
save(model, COMPLETE_MODEL_PATH)

TypeError: save() missing 1 required positional argument: 'filename'

### Test model

In [9]:
VALIDATION_DATA = "./data/plants/validation.gz"

val_data = pd.read_csv(VALIDATION_DATA)
val_data['class'] = val_data['class'].apply(lambda x: CLASSES_DICT_NAMES[x.replace("’", "")])
val_info = val_data[['label', 'class']]
X_val = val_data.drop(labels=['label', 'class'], axis=1)

# Apply the shape required
X_val = X_val.values.reshape(-1, *RESHAPE_SIZE)
Y_val = to_categorical(val_info['class'].values, num_classes=len(CLASSES_DICT_NAMES))

In [25]:
t0 = time()
repeat = int(1e1)
for k in range(repeat):
    Y_pred = model.predict(X_val)
t1 = time() - t0
print(f"Total time: {t1}\t Av time: {t1/repeat}")

Total time: 55.73194980621338	 Av: 5.573194980621338


### Quantizing Weights

In [27]:
converter = tf2.lite.TFLiteConverter.from_saved_model(MODEL_REF)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()

OSError: SavedModel file does not exist at: ./models/model2_weights_final.h5/{saved_model.pbtxt|saved_model.pb}

In [26]:
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.0, final_sparsity=0.5,
    begin_step=2000, end_step=4000)

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(
    model, pruning_schedule=pruning_schedule)

ValueError: Please initialize `Prune` with a supported layer. Layers should either be a `PrunableLayer` instance, or should be supported by the PruneRegistry. You passed: <class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'>

In [ ]:


model_for_pruning.fit(...)

Total time: 61.14887189865112	 Av: 6.1148871898651125


In [12]:
Y_pred.shape

(1384, 12)